# Mini ResNet Model

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 30
batch_size = 100
learning_rate = 0.001

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=True,
                                            transform=transform,
                                            download=True)
test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                           train=False,
                                           transform=transforms.ToTensor())
# Data Loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)



100%|██████████| 170498071/170498071 [00:02<00:00, 59116516.94it/s]


Extracting ../../data/cifar-10-python.tar.gz to ../../data/


In [3]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [4]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels, stride=stride),
                                       nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [5]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
decay = 0
model.train()
for epoch in range(num_epochs):
    
    # Decay the learning rate by a factor of 0.5 every 20 epochs
    if (epoch+1) % 20 == 0:
        decay+=1
        optimizer.param_groups[0]['lr'] = learning_rate * (0.5 ** decay)
        print(f"The new learning rate is {optimizer.param_groups[0]['lr']}")
        
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], step [{i+1}/{len(train_loader)}], Loss {loss.item()}")
        

Epoch [1/30], step [100/500], Loss 1.7886300086975098
Epoch [1/30], step [200/500], Loss 1.4154130220413208
Epoch [1/30], step [300/500], Loss 1.4515981674194336
Epoch [1/30], step [400/500], Loss 1.2049803733825684
Epoch [1/30], step [500/500], Loss 1.0533392429351807
Epoch [2/30], step [100/500], Loss 1.064954400062561
Epoch [2/30], step [200/500], Loss 1.1049596071243286
Epoch [2/30], step [300/500], Loss 0.9729300737380981
Epoch [2/30], step [400/500], Loss 1.1058732271194458
Epoch [2/30], step [500/500], Loss 0.9754817485809326
Epoch [3/30], step [100/500], Loss 0.9646298885345459
Epoch [3/30], step [200/500], Loss 0.8280516862869263
Epoch [3/30], step [300/500], Loss 1.0032823085784912
Epoch [3/30], step [400/500], Loss 0.7891208529472351
Epoch [3/30], step [500/500], Loss 0.8497903943061829
Epoch [4/30], step [100/500], Loss 0.8383074998855591
Epoch [4/30], step [200/500], Loss 0.6544916033744812
Epoch [4/30], step [300/500], Loss 0.7271671295166016
Epoch [4/30], step [400/500],

In [7]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f"Accuracy of the model on the test images: {100*correct/total}")

Accuracy of the model on the test images: 86.87
